In [14]:
import pandas as pd 
import json
import dask.dataframe as dd 

# Chargement des data sets

In [15]:
parquet_file = "../data/final_output.parquet"
data = dd.read_parquet(parquet_file)

# Traitement des données

In [16]:
def add_action_reverse(df):
    df["action_reverse"] = df.groupby("session").cumcount()
    df["action_reverse"] = df.groupby("session")["action_reverse"].transform(lambda x: x.max() - x, meta=('action_reverse', 'int32'))
    return df

def add_session_length(df):
    df['session_length'] = df.groupby('session')['session'].transform('count', meta=('session_length', 'int32'))
    return df

def add_log_recency_score(df):
    linear_interpolation = 0.1 + ((1 - 0.1) / (df['session_length'] - 1)) * (df['session_length'] - df['action_reverse'] - 1)
    df['log_recency_score'] = (2 ** linear_interpolation) - 1
    df['log_recency_score'] = df['log_recency_score'].fillna(1)
    return df

def add_type_weighted_log_recency_score(df):
    type_weights = {0: 1, 1: 6, 2: 3}
    df['type_weighted_log_recency_score'] = df['log_recency_score'] / df['type'].map(type_weights)
    return df

def apply(df, pipeline):
    if isinstance(df, (dd.DataFrame, pd.DataFrame)):
        for f in pipeline:
            df = f(df)
        return df
    else:
        raise TypeError("Input doit être un DataFrame Pandas ou Dask DataFrame")

In [18]:
pipeline = [add_action_reverse, add_session_length, add_log_recency_score, add_type_weighted_log_recency_score]

In [19]:
df_train = apply(data, pipeline)

c:\Users\eloua\AppData\Local\Programs\Python\Python313\Lib\site-packages\dask\dataframe\dask_expr\_collection.py:4208: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('type', 'float64'))

  warnings.warn(meta_warning(meta))
